In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:

# 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# datetime 컬럼을 datetime 타입으로 변환
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])


In [3]:
train['datetime']

,datetime
0,2011-01-01 00:00:00
1,2011-01-01 01:00:00
2,2011-01-01 02:00:00
3,2011-01-01 03:00:00
4,2011-01-01 04:00:00
...,...
10881,2012-12-19 19:00:00
10882,2012-12-19 20:00:00
10883,2012-12-19 21:00:00
10884,2012-12-19 22:00:00


In [4]:
test['datetime']

,datetime
0,2011-01-20 00:00:00
1,2011-01-20 01:00:00
2,2011-01-20 02:00:00
3,2011-01-20 03:00:00
4,2011-01-20 04:00:00
...,...
6488,2012-12-31 19:00:00
6489,2012-12-31 20:00:00
6490,2012-12-31 21:00:00
6491,2012-12-31 22:00:00


In [5]:
# datetime에서 유용한 특성 추출
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek

In [6]:
# 사용할 특성 선택
features = ['season', 'holiday', 'workingday', 'weather', 'temp',
            'atemp', 'humidity', 'windspeed', 'year', 'month',
            'day', 'hour', 'dayofweek']

X = train[features].values
y = train['count'].values

In [7]:
X

array([[ 1.,  0.,  0., ...,  1.,  0.,  5.],
       [ 1.,  0.,  0., ...,  1.,  1.,  5.],
       [ 1.,  0.,  0., ...,  1.,  2.,  5.],
       ...,
       [ 4.,  0.,  1., ..., 19., 21.,  2.],
       [ 4.,  0.,  1., ..., 19., 22.,  2.],
       [ 4.,  0.,  1., ..., 19., 23.,  2.]])

In [8]:
y

array([ 16,  40,  32, ..., 168, 129,  88])

In [9]:
# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# PyTorch 데이터셋 및 데이터로더 생성
class BikeDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BikeDataset(X_train, y_train)
val_dataset = BikeDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

In [11]:
# 모델 정의
class BikeDemandModel(nn.Module):
    def __init__(self):
        super(BikeDemandModel, self).__init__()
        self.fc1 = nn.Linear(len(features), 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.sigmoid(x)


In [12]:

# 모델, 손실 함수, 옵티마이저 초기화
model = BikeDemandModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(100):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

In [13]:
# 검증 데이터로 성능 평가
model.eval()
val_preds = []
with torch.no_grad():
    for X_batch, _ in val_loader:
        outputs = model(X_batch).squeeze()
        val_preds.append(outputs.numpy())

val_preds = np.concatenate(val_preds)
print('검증 데이터 RMSE:', np.sqrt(mean_squared_error(y_val, val_preds)))
print('검증 데이터 MSE:', mean_absolute_error(y_val, val_preds))

검증 데이터 RMSE: 262.75510567895805
검증 데이터 MSE: 191.0496019596566


In [14]:
# 테스트 데이터 예측
X_test = scaler.transform(test[features].values)
X_test_tensor = torch.FloatTensor(X_test)

model.eval()
with torch.no_grad():
    test_pred = model(X_test_tensor).squeeze().numpy()

# 제출 파일 생성
# submission = pd.DataFrame({
#     'datetime': test['datetime'],
#     'count': test_pred
# })
# submission.to_csv('submission_pytorch.csv', index=False)
# print('PyTorch 제출 파일이 생성되었습니다.')

PyTorch 제출 파일이 생성되었습니다.


In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# datetime 컬럼을 datetime 타입으로 변환
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

# datetime에서 유용한 특성 추출
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek

# 사용할 특성 선택
features = ['season', 'holiday', 'workingday', 'weather', 'temp',
            'atemp', 'humidity', 'windspeed', 'year', 'month',
            'day', 'hour', 'dayofweek']

X = train[features].values
y = train['count'].values

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 데이터셋 및 데이터로더 생성
class BikeDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BikeDataset(X_train, y_train)
val_dataset = BikeDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 정의
class BikeDemandModel(nn.Module):
    def __init__(self):
        super(BikeDemandModel, self).__init__()
        self.fc1 = nn.Linear(len(features), 64)  # 첫 번째 은닉층
        self.relu = nn.ReLU()                     # ReLU 활성화 함수
        self.fc2 = nn.Linear(64, 32)              # 두 번째 은닉층
        self.fc3 = nn.Linear(32, 1)               # 출력층 (활성화 함수 없음)

    def forward(self, x):
        x = self.fc1(x)                          # 첫 번째 선형층 통과
        x = self.relu(x)                         # ReLU 활성화 함수 적용
        x = self.fc2(x)                          # 두 번째 선형층 통과
        x = self.fc3(x)                          # 출력층 통과 (선형)
        return x                                 # 최종 출력 반환

# 모델, 손실 함수, 옵티마이저 초기화
model = BikeDemandModel()
criterion = nn.MSELoss()                      # 손실 함수는 평균 제곱 오차
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저

# 모델 학습
for epoch in range(100):
    model.train()                             # 학습 모드 설정
    for X_batch, y_batch in train_loader:    # 배치 단위로 데이터 로딩
        optimizer.zero_grad()                 # 기울기 초기화
        outputs = model(X_batch).squeeze()   # 모델 예측
        loss = criterion(outputs, y_batch)    # 손실 계산
        loss.backward()                       # 기울기 계산
        optimizer.step()                      # 파라미터 업데이트

# 검증 데이터로 성능 평가
model.eval()                                  # 평가 모드 설정
val_preds = []                                # 검증 데이터 예측 결과 저장 리스트
with torch.no_grad():                         # 기울기 계산을 하지 않도록 설정
    for X_batch, _ in val_loader:            # 검증 데이터 배치 단위로 로딩
        outputs = model(X_batch).squeeze()   # 모델 예측
        val_preds.append(outputs.numpy())     # 예측 결과를 리스트에 저장

val_preds = np.concatenate(val_preds)         # 리스트를 NumPy 배열로 변환
print('검증 데이터 RMSE:', np.sqrt(mean_squared_error(y_val, val_preds)))  # RMSE 계산 및 출력
print('검증 데이터 MAE:', mean_absolute_error(y_val, val_preds))            # MAE 계산 및 출력

# 테스트 데이터 예측
X_test = scaler.transform(test[features].values)  # 테스트 데이터 정규화
X_test_tensor = torch.FloatTensor(X_test)         # 텐서로 변환
model.eval()                                       # 평가 모드 설정
with torch.no_grad():                              # 기울기 계산을 하지 않도록 설정
    test_pred = model(X_test_tensor).squeeze().numpy()  # 테스트 데이터 예측

# 제출 파일 생성
submission = pd.DataFrame({
    'datetime': test['datetime'],
    'count': test_pred
})
submission.to_csv('submission_pytorch.csv', index=False)  # CSV 파일로 저장

print('PyTorch 제출 파일이 생성되었습니다.')  # 제출 파일 생성 완료 메시지

검증 데이터 RMSE: 114.97188664462664
검증 데이터 MAE: 85.84874975495211
PyTorch 제출 파일이 생성되었습니다.


In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# datetime 컬럼을 datetime 타입으로 변환
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

# datetime에서 유용한 특성 추출
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek

# 사용할 특성 선택
features = ['season', 'holiday', 'workingday', 'weather', 'temp',
            'atemp', 'humidity', 'windspeed', 'year', 'month',
            'day', 'hour', 'dayofweek']

X = train[features].values
y = train['count'].values

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 데이터셋 및 데이터로더 생성
class BikeDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = BikeDataset(X_train, y_train)
val_dataset = BikeDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 정의
class BikeDemandModel(nn.Module):
    def __init__(self):
        super(BikeDemandModel, self).__init__()
        self.fc1 = nn.Linear(len(features), 64)  # 첫 번째 은닉층
        self.relu = nn.ReLU()                     # ReLU 활성화 함수
        self.fc2 = nn.Linear(64, 32)              # 두 번째 은닉층
        self.fc3 = nn.Linear(32, 1)               # 출력층 (활성화 함수 없음)

    def forward(self, x):
        x = self.fc1(x)                          # 첫 번째 선형층 통과
        x = self.relu(x)                         # ReLU 활성화 함수 적용
        x = self.fc2(x)                          # 두 번째 선형층 통과
        x = self.relu(x)                         # ReLU 활성화 함수 적용
        x = self.fc3(x)                          # 출력층 통과
        x = torch.sigmoid(x)                     # 0과 1 사이로 변환
        return x * 100                           # 0에서 100 사이로 변환

# 모델, 손실 함수, 옵티마이저 초기화
model = BikeDemandModel()
criterion = nn.MSELoss()                      # 손실 함수는 평균 제곱 오차
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저

# 모델 학습
for epoch in range(100):
    model.train()                             # 학습 모드 설정
    for X_batch, y_batch in train_loader:    # 배치 단위로 데이터 로딩
        optimizer.zero_grad()                 # 기울기 초기화
        outputs = model(X_batch).squeeze()   # 모델 예측
        loss = criterion(outputs, y_batch)    # 손실 계산
        loss.backward()                       # 기울기 계산
        optimizer.step()                      # 파라미터 업데이트

# 검증 데이터로 성능 평가
model.eval()                                  # 평가 모드 설정
val_preds = []                                # 검증 데이터 예측 결과 저장 리스트
with torch.no_grad():                         # 기울기 계산을 하지 않도록 설정
    for X_batch, _ in val_loader:            # 검증 데이터 배치 단위로 로딩
        outputs = model(X_batch).squeeze()   # 모델 예측
        val_preds.append(outputs.numpy())     # 예측 결과를 리스트에 저장

val_preds = np.concatenate(val_preds)         # 리스트를 NumPy 배열로 변환
print('검증 데이터 RMSE:', np.sqrt(mean_squared_error(y_val, val_preds)))  # RMSE 계산 및 출력
print('검증 데이터 MAE:', mean_absolute_error(y_val, val_preds))            # MAE 계산 및 출력

# 테스트 데이터 예측
X_test = scaler.transform(test[features].values)  # 테스트 데이터 정규화
X_test_tensor = torch.FloatTensor(X_test)         # 텐서로 변환
model.eval()                                       # 평가 모드 설정
with torch.no_grad():                              # 기울기 계산을 하지 않도록 설정
    test_pred = model(X_test_tensor).squeeze().numpy()  # 테스트 데이터 예측

# 제출 파일 생성
submission = pd.DataFrame({
    'datetime': test['datetime'],
    'count': test_pred
})
submission.to_csv('submission_pytorch.csv', index=False)  # CSV 파일로 저장

print('PyTorch 제출 파일이 생성되었습니다.')  # 제출 파일 생성 완료 메시지


검증 데이터 RMSE: 199.1278539072376
검증 데이터 MAE: 123.68705905074916
PyTorch 제출 파일이 생성되었습니다.
